1. find an initialization of V s.t  SV = 0 and minimize L1 norm of V
2. improve sparsity of SV rows by combining the loss functions with given lambda, using ADMM. use first-order-cone instead of second-order cone to encourage sparsity of SV rows.

In [1]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO
using CSV
using DataFrames
using DelimitedFiles

In [2]:
vars = matread("R4T1.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
lambda = vars["lambda"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

print("done")

(2172, 4456)(4456, 50)done

In [4]:
X = zeros(size(L)[1],size(L)[2]);

n,d = size(S)

for j in 1:size(X)[2]
    model = Model(ECOS.Optimizer)
    @variable(model, L[i,j] <= x[i = 1:d] <= U[i,j])
    @constraint(model,S*x .== 0)
    @variable(model, x_abs[1:d])
    @constraint(model, x_abs .>= x)
    @constraint(model, x_abs .>= -x)
    @objective(model, Min, sum(x_abs[i] for i in 1:d));
    set_silent(model)
    optimize!(model)
    x = value.(x);
    X[:,j] = x;
    print(j)
    print(", ")
end

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 

In [5]:
n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

L = vec(L);
U = vec(U);

In [6]:
model = Model(COSMO.Optimizer)

set_optimizer_attribute(model, "max_iter",50000)
set_optimizer_attribute(model, "eps_prim_inf",1e-6)
set_optimizer_attribute(model, "eps_abs",1e-6)



@variable(model, L[i] <= x[i = 1:d*k] <= U[i],start=vec(X)[i])
@variable(model,  norm_x[1:d])

for i in 1:k
    @constraint(model,S*x[i*d-d+1:i*d] .== 0)
end

for i in 1:d
    @constraint(model,[norm_x[i]; x[[j*d-d+i for j in 1:k]]] in SecondOrderCone())
end


@objective(model,Min,sum(norm_x[i] for i in 1:d))
optimize!(model)

In [15]:
using COSMO, LinearAlgebra, SparseArrays, Test

# define new cone type
struct Nonpositives{T} <: COSMO.AbstractConvexSet{T}
    dim::Int64
end

function COSMO.project!(x::AbstractVector{T}, C::Nonpositives{T}) where {T <: AbstractFloat}
    x .= max.(x, -x)
end

using MathOptInterface, JuMP
import Base.copy
const MOI = MathOptInterface

struct FOC <: MOI.AbstractVectorSet
    dimension::Int
end
Base.copy(set::FOC) = set #this is needed for MOI

function COSMO.processSet!(b::Vector{T}, rows::UnitRange{Int}, cs, s::FOC) where {T <: AbstractFloat}
    push!(cs, Nonpositives{T}(length(rows)))
    nothing
end
#tell MOI, that COSMO supports constraints with this new cone
MOI.supports_constraint(optimizer::COSMO.Optimizer, ::Type{<:Union{MOI.VectorOfVariables, MOI.VectorAffineFunction{Float64}}}, ::Type{FOC}) = true

function COSMO.in_dual(x::AbstractVector{T}, C::Nonpositives{T}, tol::T) where {T <: AbstractFloat}
    return !any( x-> (x < -max.(x, -x)+tol), x)
end

function COSMO.in_pol_recc(x::AbstractVector{T}, C::Nonpositives{T}, tol::T) where {T <: AbstractFloat}
    return !any( x-> (x > max.(x, -x)-tol), x)
end

In [16]:
model = Model(COSMO.Optimizer)

set_optimizer_attribute(model, "max_iter",50000)
set_optimizer_attribute(model, "eps_prim_inf",1e-6)
set_optimizer_attribute(model, "eps_abs",1e-6)


@variable(model, L[i] <= x[i = 1:d*k] <= U[i],start=vec(X)[i])
@variable(model,  norm_x[1:d])
@variable(model, norm_c[1:k])

for i in 1:k
    @constraint(model,[norm_c;S*x[i*d-d+1:i*d]] in SecondOrderCone())
end

for i in 1:d
    @constraint(model,[norm_x[i]; x[[j*d-d+i for j in 1:k]]] in SecondOrderCone())
end

@objective(model,Min,sum(norm_x[i] for i in 1:d)+lambda*sum(norm_c[i] for i in 1:k))
optimize!(model)

------------------------------------------------------------------
          COSMO v0.8.1 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2021
------------------------------------------------------------------

Problem:  x ∈ R^{227306},
          constraints: A ∈ R^{783956x227306} (1654356 nnz),
          matrix size to factor: 1011262x1011262,
          Floating-point precision: Float64
Sets:     Nonnegatives) of dim: 445600
          typename(Nonpositives) of dim: 2222
          typename(Nonpositives) of dim: 2222
          typename(Nonpositives) of dim: 2222
          typename(Nonpositives) of dim: 2222
          ... and 4502 more
Settings: ϵ_abs = 1.0e-06, ϵ_rel = 1.0e-05,
          ϵ_prim_inf = 1.0e-06, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 50000,
          scaling iter = 10 (on),
          check termination every 25 iter,
          check infeasibility every 40 it

3600	1.4113e+11	1.1579e+10	1.4274e+10	7.6959e-01
3625	1.8596e+11	1.0742e+10	1.0831e+10	7.6959e-01
3650	3.3065e+11	5.8936e+09	2.6907e+10	7.6959e-01
3675	2.3150e+11	4.0565e+09	2.2553e+10	7.6959e-01


LoadError: InterruptException:

In [41]:
writedlm( "output.txt", Matrix(reshape(value.(x),(d,k))), ',')